## Importing libraries

In [1]:
import pandas
import keras

In [2]:
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential

In [3]:
#from google.colab import drive

In [4]:
#drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
finaldata=pd.read_csv('/content/drive/My Drive/project/data.csv')

In [6]:
finaldata['target']=finaldata['target'].replace({'NoHate':0,'Hate':1})

In [7]:
finaldata.shape

(187902, 11)

In [8]:
finaldata.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'comment_text', 'target', 'Num_words',
       'words', 'clean_comment', 'clean_comment_wo_stop',
       'clean_comment_lemmatized', 'clean_comment_string',
       'clean_comment_string1'],
      dtype='object')

In [9]:
finaldata.sample(2)

,Unnamed: 0.1,Unnamed: 0,comment_text,target,Num_words,words,clean_comment,clean_comment_wo_stop,clean_comment_lemmatized,clean_comment_string,clean_comment_string1
1903,1903,1903,Summaries \n\nSorry bout that... I'll try to r...,0,14,"['Summaries', 'Sorry', 'bout', 'that...', ""I'l...",summaries sorry bout that i ll try to rem...,summaries sorry bout try remember better mahal...,summaries sorry bout try remember better mahal...,summaries sorry bout try remember better mahal...,summaries sorry bout try remember better mahal...
35933,35933,35933,"""\n\n \n\nHi. Could you comment on his unbloc...",0,24,"['""', 'Hi.', 'Could', 'you', 'comment', 'on', ...",hi could you comment on his unblock re...,hi could comment unblock request like attitude...,hi could comment unblock request like attitude...,hi could comment unblock request like attitude...,hi could comment unblock request like attitude...


In [10]:
textdata=finaldata['clean_comment_string1']
target=finaldata['target']

In [11]:
X_train,X_test,Y_train,Y_test=train_test_split(textdata,target,test_size=0.25,random_state=12,stratify=target)

In [12]:
X_train.shape

(140926,)

In [13]:
Y_train.value_counts()

0    118563
1     22363
Name: target, dtype: int64

In [29]:
X_train1=X_train[:1000]
Y_train1=Y_train[:1000]
X_test1=X_test[:100]
Y_test1=Y_test[:100]

In [14]:
Y_train

20444     0
105646    0
108946    0
131182    0
20283     0
         ..
134893    0
28168     0
73295     0
165826    0
35594     0
Name: target, Length: 140926, dtype: int64

In [30]:
X_train1

20444     undiscussed copypaste move alam road recently ...
105646    like example however murder mentioned opening ...
108946    funk rock per guidelines funk metal subgenre f...
131182    think confused something nobody executed july ...
20283     media media example ogg media media example og...
                                ...                        
50731     think som bitch fuckin nuts mean fuck cares in...
38764     sorry first foray wp problematic disappointing...
93478     sick yeti animatronic yeti frozen place years ...
108326    warning given warning really really balancing ...
139740    relatively obscure details background book cre...
Name: clean_comment_string1, Length: 1000, dtype: object

In [15]:
def tokenizer(X_train, X_test,maxword=75000,sequ=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0).astype('str')
    text = np.array(text)
    tokenizer = Tokenizer(num_words=maxword)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=sequ)
    indices = np.arange(text.shape[0])
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/content/drive/My Drive/project/glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    return (X_train, X_test, word_index,embeddings_index)

In [16]:
train_Glove,test_Glove, wordindex,embeddingsindex = tokenizer(X_train,X_test)

(187902, 50)


In [17]:
def LSTMCNN(wordi, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50):
    kernel_size = 2
    filters = 64
    pool_size = 2
    gru_node = 256
    embedding_matrix = np.random.random((len(wordi) + 1, EMBEDDING_DIM))
    for word, i in wordi.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("EMBEDDING_DIM is not equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector
    model = Sequential()
    model.add(Embedding(len(wordi) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [18]:
modelrcnnlstm = LSTMCNN(wordindex,embeddingsindex, 3)

In [19]:
modelrcnnlstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            8843350   
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 49, 64)            6464      
                                                                 
 max_pooling1d (MaxPooling1  (None, 24, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 64)            8256      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 11, 64)            0         
 g1D)                                                   

In [48]:
modelrcnnlstm.fit(train_Glove,Y_train,epochs=10,batch_size=128,verbose=2)

Epoch 1/10
1101/1101 - 67s - loss: 0.2069 - accuracy: 0.9158 - 67s/epoch - 61ms/step
Epoch 2/10
1101/1101 - 61s - loss: 0.1956 - accuracy: 0.9204 - 61s/epoch - 55ms/step
Epoch 3/10
1101/1101 - 61s - loss: 0.1873 - accuracy: 0.9245 - 61s/epoch - 55ms/step
Epoch 4/10
1101/1101 - 61s - loss: 0.1785 - accuracy: 0.9279 - 61s/epoch - 55ms/step
Epoch 5/10
1101/1101 - 67s - loss: 0.1709 - accuracy: 0.9312 - 67s/epoch - 61ms/step
Epoch 6/10
1101/1101 - 60s - loss: 0.1634 - accuracy: 0.9346 - 60s/epoch - 55ms/step
Epoch 7/10
1101/1101 - 60s - loss: 0.1571 - accuracy: 0.9375 - 60s/epoch - 55ms/step
Epoch 8/10
1101/1101 - 61s - loss: 0.1512 - accuracy: 0.9390 - 61s/epoch - 56ms/step
Epoch 9/10
1101/1101 - 60s - loss: 0.1462 - accuracy: 0.9417 - 60s/epoch - 54ms/step
Epoch 10/10
1101/1101 - 65s - loss: 0.1412 - accuracy: 0.9436 - 65s/epoch - 59ms/step


In [21]:
hate_prediction=modelrcnnlstm.predict(test_Glove)

1468/1468 [==============================] - 10s 6ms/step


In [22]:
predict_class = np.argmax(hate_prediction, axis=1)

In [23]:
predict_class

array([0, 1, 1, ..., 0, 0, 1])

BERT TOEKNIZER

In [25]:
#!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


In [26]:
#from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

In [27]:
berttokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [31]:
#Specify maximum sequence length to pad or truncate
max_len = 0

for seq in X_train1:
    # Tokenize the text by BERT tokenizer
    input_ids = berttokenizer.encode(seq, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Maximum sequence length', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (764 > 512). Running this sequence through the model will result in indexing errors


Maximum sequence length 764


In [32]:
# Tokenize all of the sequences and map the tokens to thier IDs.
input_ids_train = []
attention_masks_train = []

# For every sequences
for seq in X_train1:
    encoded_dict = berttokenizer.encode_plus(
                        seq,                             # Sequence to encode
                        add_special_tokens = True,       # Add '[CLS]' and '[SEP]'
                        max_length = 128,
                        padding = 'max_length',          # Pad and truncate
                        truncation=True,                 #Truncate the seq
                        return_attention_mask = True    # Construct attn. masks
                                 # Return pytorch tensors
                   )

    # Add the encoded sequences to the list
    input_ids_train.append(encoded_dict['input_ids'])

    # And its attention mask
    attention_masks_train.append(encoded_dict['attention_mask'])

In [33]:
# Tokenize all of the sequences and map the tokens to thier IDs.
input_ids_test = []
attention_masks_test = []

# For every sequences
for seq in X_test1:
    encoded_dict_test = berttokenizer.encode_plus(
                        seq,                             # Sequence to encode
                        add_special_tokens = True,       # Add '[CLS]' and '[SEP]'
                        max_length = 128,
                        padding = 'max_length',          # Pad and truncate
                        truncation=True,                 #Truncate the seq
                        return_attention_mask = True    # Construct attn. masks
                                 # Return pytorch tensors
                   )

    # Add the encoded sequences to the list
    input_ids_test.append(encoded_dict_test['input_ids'])

    # And its attention mask
    attention_masks_test.append(encoded_dict['attention_mask'])

In [34]:
import tensorflow as tf

In [35]:
input_ids=tf.convert_to_tensor(input_ids_train)
attention_masks=tf.convert_to_tensor(attention_masks_train)

In [36]:
input_ids_test=tf.convert_to_tensor(input_ids_test)
attention_masks_test=tf.convert_to_tensor(attention_masks_test)

In [37]:
train_target=tf.convert_to_tensor(Y_train1)

In [38]:
test_target=tf.convert_to_tensor(Y_test1)

In [39]:
input_ids.shape

TensorShape([1000, 128])

In [40]:
from transformers import BertTokenizer, TFBertForSequenceClassification

In [41]:
bert_model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
model_path="/content/drive/My Drive/project/bert_model.h5"
log_path="/content/drive/My Drive/project"

In [43]:
callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath=model_path,save_weights_only=True,monitor="val_loss",mode="min",save_best_only=True),tf.keras.callbacks.TensorBoard(log_dir=log_path)]
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

In [44]:
bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metrics])

In [49]:
history=bert_model.fit([input_ids,attention_masks],train_target,batch_size=32,epochs=10,validation_data=([input_ids_test,attention_masks_test],test_target),callbacks=callbacks)

Epoch 1/10
32/32 [==============================] - 28s 869ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.4335 - val_accuracy: 0.8900
Epoch 2/10
32/32 [==============================] - 26s 822ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.4443 - val_accuracy: 0.9100
Epoch 3/10
32/32 [==============================] - 26s 827ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.4564 - val_accuracy: 0.9100
Epoch 4/10
32/32 [==============================] - 27s 831ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.4690 - val_accuracy: 0.9100
Epoch 5/10
32/32 [==============================] - 27s 846ms/step - loss: 0.0085 - accuracy: 0.9990 - val_loss: 0.4682 - val_accuracy: 0.9000
Epoch 6/10
32/32 [==============================] - 26s 824ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.4954 - val_accuracy: 0.9300
Epoch 7/10
32/32 [==============================] - 26s 828ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.4567 - val_accuracy: 0.9200